pandas -> CSV-Datein einlesen;
spacy -> Satzzeichen und Stopwörter;
collections/Counter -> Häufigkeitsverteilung der Wörter;

In [7]:
import pandas as pd
import spacy
from collections import Counter

Einlesen des Datensatzes und Definierung der Stopwörter (muss noch erweiter werden auf alle unnötigen Stopwörter)

In [8]:
data = pd.read_csv("20 newsgroups/20newsgroups.csv", on_bad_lines='skip', sep=';')
data = data.iloc[:,1:3]

doppelte_zeilen = data.duplicated().sum()
na_zeilen = data.isna().sum()
data = data.dropna()
data = data.drop_duplicates()
print("Anzahl doppelter Zeilen:", doppelte_zeilen)
print("Anzahl null Zeilen:", na_zeilen)


nlp = spacy.load("en_core_web_sm")


Anzahl doppelter Zeilen: 89
Anzahl null Zeilen: text     90
group     0
dtype: int64


Erstellung der Häufigkeitsverteilung, mit gleichzeitiger Entfernung von Satzzeichen/Stopwörtern, und Ausgabe der 20 häufigsten Wörter (Ausführung auch hier nicht notwendig, dauert 2min)

In [13]:
word_freq = Counter()

for text in data["text"]:
    doc = nlp(text)  # NLP-Pipeline auf den Text anwenden
    
    # Filter und Lemmatisierung in einem Schritt
    filtered_words = [
        token.lemma_.lower() 
        for token in doc 
        if not token.is_stop and not token.is_punct and token.is_alpha
    ]
    
    # Aktualisiere die Häufigkeitsverteilung
    word_freq.update(filtered_words)

# Ausgabe der Top 20 Wörter
print("Top 20 häufigste Wörter:")
for word, count in word_freq.most_common(20):
    print(f"{word}: {count}")


Top 20 häufigste Wörter:
image: 1816
know: 1408
space: 1361
god: 1301
think: 1242
like: 1233
people: 1223
file: 1119
time: 1090
system: 1023
say: 966
use: 937
good: 933
program: 906
find: 861
point: 823
thing: 821
jpeg: 770
way: 768
look: 759


n most common words

In [14]:
print(word_freq.most_common(1000))
name_list_length = 1000
top_n_words = [word for word, _ in word_freq.most_common(name_list_length)]

[('image', 1816), ('know', 1408), ('space', 1361), ('god', 1301), ('think', 1242), ('like', 1233), ('people', 1223), ('file', 1119), ('time', 1090), ('system', 1023), ('say', 966), ('use', 937), ('good', 933), ('program', 906), ('find', 861), ('point', 823), ('thing', 821), ('jpeg', 770), ('way', 768), ('look', 759), ('believe', 729), ('work', 709), ('want', 678), ('read', 668), ('need', 666), ('format', 657), ('year', 654), ('bit', 652), ('include', 651), ('available', 646), ('come', 645), ('new', 644), ('try', 618), ('post', 605), ('mean', 604), ('write', 596), ('jesus', 583), ('support', 582), ('datum', 573), ('book', 564), ('right', 559), ('question', 558), ('software', 556), ('launch', 554), ('information', 550), ('color', 546), ('go', 542), ('problem', 527), ('graphic', 519), ('version', 509), ('base', 503), ('get', 501), ('send', 499), ('etc', 494), ('tell', 485), ('well', 468), ('day', 468), ('line', 462), ('earth', 461), ('life', 460), ('true', 455), ('religion', 449), ('high'

Erstellung der neuen CSV-Datei

In [15]:
wort_namen_list = [wort for wort, _ in word_freq.most_common(name_list_length)]
wort_namen_list.append('groupID')
processedData = pd.DataFrame(columns=wort_namen_list)
processedData.to_csv("20newsgroups_processed.csv", index=False)
processedData.head()

,image,know,space,god,think,like,people,file,time,system,...,ability,parallel,technique,bright,modern,commit,galaxy,previous,potential,groupID


Gesamte CSV-Datei (dauert fast 3min --> CSV-Datei ist schon erstellt, ausführen nicht notwendig)

In [ ]:
# Liste zum Speichern der Ergebnisse
rows = []

for index, row in data.iterrows():
    text = str(row['text'])
    group = str(row['group'])

    doc = nlp(text)
    count_in_top_n = Counter(
        token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop and not token.is_punct
    )

    haeufigkeiten = {word: count_in_top_n.get(word, 0) for word in top_n_words}
    haeufigkeiten['groupID'] = group
    rows.append(haeufigkeiten)

# Erstelle einen DataFrame aus der Liste der Ergebnisse
processedData = pd.DataFrame(rows)

# Speichere den DataFrame und gib die ersten Zeilen aus
processedData.to_csv("20newsgroups_processed.csv", index=False)
print(processedData.head())

Einfache Ausgabe

In [74]:
processedData.head()

,,=,,>,,space,,don\'t,people,like,...,media,charge,washington,let\,moment,published,fairly,refer,child,groupID
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
